
# What does 1GRDvs64 SIR_HIST_SD do

## Things we should do with this or a new related notebook

** Note that the calculation of the spatial standard deviation is ONLY done in this notebook, not the map, but if we rewrite, we should make it so that the area that can be calculated is flexible (the GRD/64SIR extent, the 8 around the SIR pixel of interest)

** clean up notebooks to allow for separately doing (some of) Separate time series, multi plots, histograms, export of data , test breaks related to if you haven't run the prior cells]

** Use standardized filenames for pkl/tif files

** If we are reading in the whole cube, then could we change the points after the read in (but before the DAV, SD calcs, or just different pixel ids) so that we could more efficiently process subsets 

** Some of the original maps don't work and need updating. I think they are kind of useful for quick looks,

Things we should do with this or a new related notebook

**Build this to use the pkl files from Large_Melt_Map. This would allow less processing time and less duplication. It would also allow us to do these things for more pixels and areas at a time, which would increase the exploration piece. 



**Note that the calculation of the spatial standard deviation is ONLY done in this notebook, not the map, but if we rewrite, we should make it so that the area that can be calculated is flexible (the GRD/64SIR extent, the 8 around the SIR pixel of interest)



**clean up notebooks to allow for separately doing (some of) Separate time series, multi plots, histograms, export of data , test breaks related to if you haven't run the prior cells]

**It would be good to standardize the filenames for export so that the sitename goes into the filename so we don't overwrite and mix up the output; MJ added threshold, count, window, cubesubset to filenames in large meltmap
e.g. have output depend on the pkl file input (add a label, hist etc)
new workflow could be to start with the pkl file/cube/site e.g.
also think about what we can read in bcs this is working w raw data (TB), SD, SSD, DAV etc.

**If we are reading in the whole cube, then could we change the points after the read in (but before the DAV, SD calcs, or just different pixel ids) so that we could more efficiently process subsets
**Some of the original maps don't work and need updating. I think they are kind of useful for quick looks, 


In [ ]:
%matplotlib inline
import glob
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.basemap import Basemap
from netCDF4 import Dataset, num2date
import numpy as np
import os
if os.name == 'nt':
    os.environ["PROJ_LIB"] = os.path.join(os.environ["CONDA_PREFIX"], "Library", "share")
    os.environ["GDAL_DATA"] = os.path.join(os.environ["CONDA_PREFIX"], "Library", "share", "gdal")
import pandas as pd
from pathlib import Path
import re

from cetbtools.ease2conv import Ease2Transform

# the set_trace() command is helpful for entering interactive
# debugging mode to step through lines of code in this notebook
from IPython.core.debugger import set_trace

In [ ]:
# Set pandas display options to display entire DataFrame, use None for all 
pd.set_option('display.max_rows', 500)
pd.set_option('display.min_rows', 120)
pd.set_option('display.max_columns', 20)
pd.set_option('display.width', 120)
pd.set_option('display.max_colwidth', 10)

In [ ]:
# Define the local machine location of CETB data cubes
# This directory is expected to contain subdirectories in the following hierarchy
# that duplicates the hierarchy on the Google Shared Drive NSIDC-SD-CETB/v1/, 
# for example:
# dataDir/F13_SSMI/N/nc_cubes/cubes_<regionName>

#JR notes setting up for Lunga, not sure if this is most recent version

user = 'MAHMac' #Mahboubeh #Mariah #MJWindows
if ('Joan' == user):
    dataDir = '/home/jmr204/rdrive/jmr204group/CETB_cubes/' #jmr lunga #'/mnt/data3/cetb/nsidc0630_v1/' #jmr machine fringe 
    scriptDir = '/home/jmr204/ipynb_melt_onset/scripts'#lunga #'/mnt/data3/cetb/ipynb_melt_onset/scripts' #fringe
    outDir = '/home/jmr204/cetb/ipynb_melt_onset_plots'#'/mnt/data3/cetb/ipynb_melt_onset_plots'
    outNoisy = Path(Path.home(), 'noisy')
elif ('Mariah' == user):
    dataDir = Path(Path.home(), 'nsidc0630_v1') # Mariah's PC or Mary Jo's Mac
    scriptDir = Path(Path.home(), 'ipynb_melt_onset', 'scripts')
    outDir = Path(Path.home(), 'ipynb_melt_onset_plots') 
    outNoisy = Path(Path.home(), 'noisy')
elif ('MJWindows' == user):
    dataDir = Path('Z:/mj On My Mac/nsidc0630_v1') # Mary Jo's Windows machine
    scriptDir = Path(Path.home(), 'ipynb_melt_onset', 'scripts')
    outDir = Path(Path.home(), 'ipynb_melt_onset_plots')
    outNoisy = Path(Path.home(), 'noisy')
elif ('MJMac' == user):
    dataDir = Path(Path.home(), 'nsidc0630_v1') # Mary Jo's Mac
    scriptDir = Path(Path.home(), 'ipynb_melt_onset', 'scripts')  
    outDir = Path(Path.home(), 'nsidc0630_v1') 
    outNoisy = Path(Path.home(), 'nsidc0630_v1', 'noisy')
elif ('MAHMac' == user):
    dataDir = Path(Path.home(), 'nsidc0630_v1') # Mary Jo's Mac
    scriptDir = Path(Path.home(), 'Projects', 'ipynb_melt_onset', 'scripts')  
    outDir = Path(Path.home(), 'nsidc0630_v1') 
    outNoisy = Path(Path.home(), 'nsidc0630_v1', 'noisy')
elif ('Mahboubeh' == user):
    dataDir = Path('Z:/mj On My Mac/nsidc0630_v1') # Mary Jo's Windows machine
    scriptDir = Path(Path.home(), 'ipynb_melt_onset', 'scripts')
    outDir = Path(Path.home(), 'ipynb_melt_onset_plots')
    outNoisy = Path(Path.home(), 'noisy')
else:
    raise ValueError("unknown user= %s\n" % (user) )
    
# Make the output directory for plot images, if it doesn't yet exist
if not os.path.exists(outDir):
    os.makedirs(outDir)
    
if not os.path.exists(outNoisy):
    os.makedirs(outNoisy)
    
noisydirs = ['sd', 'noisytb_csv', 'noisydav_csv', 'noisytbSIR_csv', 'noisydavSIR_csv']
for dir in noisydirs:
    nextDir = Path(outNoisy, dir)
    if not os.path.exists(nextDir):
        os.makedirs(nextDir)
    
%cd $scriptDir
dataDir, user

In [ ]:
noisydirs = ['sd', 'noisytb_csv', 'noisydav_csv', 'noisytbSIR_csv', 'noitydavSIR_csv']
for dir in noisydirs:
    print('dir=%s' % dir)
    

In [ ]:
from CETB_IO import read_Tb
from CETB_IO import read_Tb_std_dev
from CETB_IO import coords
from CETB_IO import read_Tb_whole
from CETB_IO import find_cube_offset
from CETB_IO import grid_locations_of_subset
from CETB_IO import years_for
from CETB_IO import get_sir_info
from CETB_IO import get_site_boundaries
from CETB_algorithms import DAV_MOD
from CETB_algorithms import XPGR
from CETB_algorithms import calc_DAV
from CETB_algorithms import end_high_DAV
from CETB_algorithms import D_DAV
from CETB_algorithms import Winter_DAV
from CETB_analysis import Tb_hist_annual
from CETB_analysis import DAV_hist_annual
from CETB_analysis import Tb_hist_monthly
from CETB_analysis import DAV_hist_monthly
from CETB_analysis import TbDAV_series_one_year
from CETB_analysis import early_melt_events
from CETB_analysis import min_max_series
from CETB_analysis import MOD_array

## Specify inputs

In [ ]:
get_site_boundaries('')

In [ ]:
#Specify region, satellite, sensor, channel, and image reconstruction algorithm of interest in file name
# this notebook will read in 2 CETB datasets so that channels/algorithms/sensors can be compared
region='AKYukon'  #make this the same syntax as cubefilenames and sub-directory #GLaIL #AKYukon CEurope
sat_GRD='F18'   #'GCOMW1' #'AQUA' for AMSRE, 'F13','F14','F15'... for SSMI
sat_SIR= 'F18'
sensor_GRD='SSMIS'  #'AMSRE', 'SSMI', 'SSMIS' 'AMSR2' etc.
sensor_SIR='SSMIS'
channel_GRD='37V'  #'36V','36H', '18V','18H', etc. '19V','19H' and '37V','37H' for SSMI)
channel_SIR='37V'
alg_GRD='GRD'   #SIR or GRD
alg_SIR='SIR'

hemName = 'N'   

# get sir to grd factor and sir_gpd name
sir_2_grd_factor, sir_gpd = get_sir_info(channel_SIR, hem=hemName)

cubeType_GRD = channel_GRD + '-' + alg_GRD
cubeType_SIR = channel_SIR + '-' + alg_SIR
  
if ('SSMI' == sensor_GRD) or ('SSMIS' == sensor_GRD):
    provider='CSU' 
    version='v1.*'
elif ('AMSR2' ==sensor_GRD):
    provider='PPS_XCAL' #need to check this
    version='v1.1'
elif 'AMSRE' == sensor_GRD:
    provider='RSS'
    version='v1.3'

Years_all=years_for(sat_GRD)
#might want to truncate Years to subset if very slow during testing
#if we give it more years than available what do we want it to do? 
#warn me but return what it finds
    
# either change to the directory where the data is or specify it
datadir_GRD = "%s/%s_%s/%s/nc_cubes/cubes_%s" % (
    dataDir, sat_GRD, sensor_GRD, hemName, region)
datadir_SIR = "%s/%s_%s/%s/nc_cubes/cubes_%s" % (
    dataDir, sat_SIR, sensor_SIR, hemName, region)


# filepath patterns for each type of cubefile
prefix_GRD = "CETB.cubefile.%s.%s_%s-%s-%s-%s-%s" % (
    region, sat_GRD, sensor_GRD, channel_GRD, alg_GRD, provider, version)
prefix_SIR = "CETB.cubefile.%s.%s_%s-%s-%s-%s-%s" % (
    region, sat_SIR, sensor_SIR, channel_SIR, alg_SIR, provider, version)

#Get lat/lon boundaries and SiteLabel for plots from get_site_boundaries
#Enter a label with no punctuation or spaces for filenames

#Site='114 km South of Barrow airport, AK' #114 km south of airport to get an inland site
SiteLabel='Barrow114kmS'
#SiteLabel='NEUkrain'
lat_start, lat_end, lon_start, lon_end, Site=get_site_boundaries(SiteLabel)
lat_start, lat_end, lon_start, lon_end, Site

In [ ]:
# check that the GRD filename prefix is really what we have on the local machine
datadir_GRD, prefix_GRD

## Get row/col coordinates of cells to examine

Find the N25 (row, col) coordinates that correspond to our site coordinates

Convert this single GRD cell coordinate to the envelope of SIR cell (row, col) coordinates


In [ ]:
rows_cols_GRD = coords(datadir_GRD, prefix_GRD, lat_start, lat_start, lon_start, lon_start)
rows_cols_env = tuple(np.array(rows_cols_GRD) * sir_2_grd_factor)
print(rows_cols_GRD)
print(rows_cols_env)


In [ ]:
datadir_SIR, prefix_SIR

In [ ]:
# build the string that is the index to the SIR cell closest to our site
# note that this is likely inside the SIR cell envelope
rows_cols = coords(datadir_SIR, prefix_SIR, lat_start, lat_start, lon_start, lon_start)
Site_nearest_row_col = "%s,%s" % (str(rows_cols[0]),str(rows_cols[2])) 
Site_nearest_row_col

In [ ]:
# check that the directory, filename pattern and Years are what we expect
# subset Years to a shorter set if only testing, because this takes a long time
subYears = Years_all[:2]
datadir_SIR, prefix_SIR, subYears

In [ ]:
#new cell to read whole dataset
#takes a long time, for testing substitute a year for Years
# See implementation of read_Tb_whole in the scripts directory for what is passed back
data_SIR = read_Tb_whole(datadir_SIR, prefix_SIR, subYears,
                         rows_cols_env[0], rows_cols_env[1], rows_cols_env[2], rows_cols_env[3])

data_GRD = read_Tb_whole(datadir_GRD, prefix_GRD, subYears, 
                         rows_cols_GRD[0], rows_cols_GRD[1], rows_cols_GRD[2], rows_cols_GRD[3])

In [ ]:
data_SIR['latitude'].data


In [ ]:
# calculate DAV for the Tb data that was imported
DAV_GRD = calc_DAV(data_GRD['TB'])
DAV_SIR = calc_DAV(data_SIR['TB'])

# New output from MOD_array

MOD_DOY_array: this is a masked array (used to be the only thing Mitch returned), contains doy of average melt over all years for each pixel. It is now a single column in MOD_DOY_df.

MOD_DOY_df: MJB: I think this is more useful, it is a data frame with 1 row for each pixel, and 1 column for each year, plus extra columns for the 'Avg' 'Max' and 'Min'. Contents of the data are doy of first time melt criteria were met. 'Avg' column is the contents of the old MOD_DOY_array. There are also columns with geolocation for each pixel x,y, row, col, lat, lon

meltflag_df: This may also be useful for further analysis, 1 row for each date (2 dates per day), 1 column for each pixel. Values are 1 if DAV/TB thresholds were exceeded on this date, 0 otherwise.

In [ ]:
#set up MOD parameters
# can tweak thresholds and count/window, legacy algorithm used 
# 3 occurrences in 5 day window and shows '3' (count) with the first pixel in the subset to melt because of the counter
#3 out of5 days is common for lower latitudes or intermittent melt (But not too intermittent)

# From Matias Fall 2022 forward facing indexer to assign rolling sum value to the beginning of window
# Setting the window_size sets the number of observations,'14' would be 7 days (2 measurements per day)
# If you don't want to use the forward facing indexer, then change "window" to a numeral, this will assign
# the rolling sum value to the end of the window
MOD_window = 1 # for 5 days, change this to 10
MOD_count = 1 # for 3 occurrences change this to 3
indexer = pd.api.indexers.FixedForwardWindowIndexer(window_size=MOD_window)
window = indexer

EHD_window = 1
EHD_count = 1
if ('SSMI' == sensor_GRD) or ('SSMIS' == sensor_GRD):
    DAV_threshold=10 
    Tb_threshold=247
elif 'AMSRE' == sensor_GRD:
    DAV_threshold=18 
    Tb_threshold=252
elif 'AMSR2' == sensor_GRD:
    DAV_threshold=18 
    Tb_threshold=252

In [ ]:
# get array of MODs for each pixel in the 64 set to use for map
# This will contain each year and the Avg MOD for all years
# calculate melt onset date, each row returned is the MOD for that year, 
MOD_DOY_df, meltflag_df, EHD_DOY_df, EHDflag_df = MOD_array(
    datadir_SIR, prefix_SIR, data_SIR, DAV_SIR, rows_cols_env, 
    subYears, window, MOD_count, EHD_window, EHD_count, DAV_threshold, Tb_threshold)
MOD_DOY_df

In [ ]:
# MOD of the GRD pixel 
# Use the same thresholds/criteria as for SIR
MOD_DOY_GRD_df, meltflag_GRD_df, EHD_DOY_GRD_df, EHDflag_GRD_df = MOD_array(
    datadir_GRD, prefix_GRD, data_GRD, DAV_GRD, rows_cols_GRD, 
    subYears, window, MOD_count, EHD_window, EHD_count, DAV_threshold, Tb_threshold)
MOD_DOY_GRD_df

In [ ]:
# Trial-and-error decide on size for GRD vs. SIR scatter markers
#np.sqrt((1200)/8)^2
sSIR = 100
sGRD = 3200

In [ ]:
outFile = Path(outDir, "%s.png" % SiteLabel)

# PLOT the MOD MAP - avg of all years or one year of interest
# Decide here:
#plot_type='all' # DO THIS to show avg of all years
# OR
plot_type='year' # DO THIS and choose a year for 1 year of interest
year=2010
## fixme ## the scale bar does not work for single year (values range -.1 -+.1)

if plot_type == 'all':
    SIR_array = MOD_DOY_df['Avg'].values
    GRD_array = MOD_DOY_GRD_df['Avg'].values
    label = 'Avg DOY ' + str(Years[0]) + '-' + str(Years[-1])
    title = sensor_SIR + '-' + Site + ' - MOD - (' + label + ')'
elif plot_type == 'year':
    SIR_array = MOD_DOY_df[year].values
    GRD_array = MOD_DOY_GRD_df[year].values
    label = str(year) + 'DOY' 
    title = sensor_SIR + '-' + Site + ' - MOD - (' + label + ')'

# Set a few common things
#min day of year to plot--look at both data arrays and subtract 2 days
minday = np.amin([np.amin(GRD_array), np.amin(SIR_array)]) - 2
#max day of year to plot same here but add 2 days
maxday = np.amax([np.amax(GRD_array), np.amax(SIR_array)]) + 2

graticule_fontsize = 4

fig, axes = plt.subplots(1, 2, figsize=(10,8))

axes[0].set_title(title)
# create polar stereographic Basemap instance.
m = Basemap(ax=axes[0], projection='stere',lon_0=lon_start,lat_0=lat_start,lat_ts=90.,\
            llcrnrlat=(lat_start-.7),urcrnrlat=(lat_start+.7),\
            llcrnrlon=(lon_start-.7),urcrnrlon=(lon_start+.7),\
            rsphere=6371200.,resolution='l',area_thresh=10000, epsg=3857)
m.drawcoastlines()
m.drawcountries()
m.drawstates()
#m.etopo(scale=5, alpha=0.7)
#m.bluemarble(scale=1)
parallels = np.arange(0.,90,0.5)
m.drawparallels(parallels,labels=[1,0,0,0],fontsize=10)
# draw meridians
meridians = np.arange(180.,360.,0.5)
m.drawmeridians(meridians,labels=[0,0,0,1],fontsize=10)
m.arcgisimage(service='World_Physical_Map', xpixels = 1500, verbose= True)

x,y = m(MOD_DOY_df['longitude'].values, MOD_DOY_df['latitude'].values)
#m.plot(x,y, 'bs', markersize=11, markeredgewidth=1, alpha=0.2)
im0 = m.scatter(x, y, c=SIR_array, s=sSIR, marker='s', lw=0, cmap='BuPu_r', alpha=.8,
                vmin=minday, vmax=maxday)

# Get the current axes and configure placement so colorbar will
# be 5% of ax width and the padding will be 0.08 in.
divider = make_axes_locatable(axes[0])
cax = divider.append_axes("right", size="5%", pad=0.08)
cbar = plt.colorbar(im0, cax=cax, label='DOY')

# plot a red line on colorbar for the MOD for the GRD pixel
cbar.ax.hlines(cbar.norm(GRD_array[0]), 0, 1, color='green') 

# create polar stereographic Basemap instance.
m = Basemap(ax=axes[1], projection='stere',lon_0=lon_start,lat_0=lat_start,lat_ts=90.,\
            llcrnrlat=(lat_start-.7),urcrnrlat=(lat_start+.7),\
            llcrnrlon=(lon_start-.7),urcrnrlon=(lon_start+.7),\
            rsphere=6371200.,resolution='l',area_thresh=10000, epsg=3857)
m.drawcoastlines()
m.drawcountries()
m.drawstates()
#m.etopo(scale=5, alpha=0.7)
#m.bluemarble(scale=1)
parallels = np.arange(0.,90,0.5)
m.drawparallels(parallels,labels=[1,0,0,0],fontsize=10)
# draw meridians
meridians = np.arange(180.,360.,0.5)
m.drawmeridians(meridians,labels=[0,0,0,1],fontsize=10)
m.arcgisimage(service='World_Physical_Map', xpixels = 1500, verbose= True)

x,y = m(MOD_DOY_GRD_df['longitude'].values, MOD_DOY_GRD_df['latitude'].values)
#m.plot(x,y, 'bs', markersize=11, markeredgewidth=1, alpha=0.2)
im1 = m.scatter(x, y, c=GRD_array, s=sGRD, marker='s', lw=0, cmap='BuPu_r', alpha=.8,
                vmin=minday, vmax=maxday)          

# Get the current axes and configure placement so colorbar will
# be 5% of ax width and the padding will be 0.05 in.
divider = make_axes_locatable(axes[1])
cax = divider.append_axes("right", size="5%", pad=0.05)
cbar = plt.colorbar(im1, cax=cax, label='DOY')

plt.tight_layout(pad=3.0)
plt.savefig(outFile)
plt.show()

print("figure saved to %s\n" % outFile)

In [ ]:
# load SIR Tb standard deviation data from netCDF file with function
Tb_std_dev=read_Tb_std_dev(datadir_SIR, prefix_SIR, subYears,
                           rows_cols_env[0],rows_cols_env[1],rows_cols_env[2],rows_cols_env[3])
#Tb_std_dev

In [ ]:
#need to run this cell for the max DAV to work
y_dims_list=list(range(len(data_SIR['TB'][0,:,0])))  # creates a list of the y-dimension pixel indices, used for plotting
x_dims_list=list(range(len(data_SIR['TB'][0,0,:])))	# creates a list of the x-dimension pixel indices, for plotting
y_s=list(range(rows_cols_env[0],rows_cols_env[1]))  # makes a list of the y(row) numbers so the dataframe of early melt events can be indexed
x_s=list(range(rows_cols_env[2],rows_cols_env[3]))  # makes a list of the x(col) numbers so the dataframe of early melt events can be indexed

# create dataframe of the 64 SIR pixels
Tb_std_frame=pd.DataFrame()
for i in y_dims_list:
    for j in x_dims_list:
        column=pd.DataFrame(data=Tb_std_dev[:,i,j], columns=[str(y_s[i])+','+str(x_s[j])])
        Tb_std_frame=pd.concat([Tb_std_frame,column],axis=1)
Tb_std_frame.index=data_SIR['cal_date']
Tb_std_frame

In [ ]:
# load dataframe of Tb GRD for plotting Tb on specified day and observation
Tb_std_dev_GRD = read_Tb_std_dev(datadir_GRD, prefix_GRD, subYears,
                                 rows_cols_GRD[0],rows_cols_GRD[1],rows_cols_GRD[2],rows_cols_GRD[3])
Tb_std_dev_GRD

In [ ]:
# slice of dataframe described above
Tb_sd_frame_GRD=pd.DataFrame(Tb_std_dev_GRD[:,0,0], index=data_GRD['cal_date'])
Tb_sd_frame_GRD

In [ ]:
# load dataframe of Tb GRD for plotting Tb on specified day and observation
Tb_frame_GRD = pd.DataFrame(data_GRD['TB'][:,0,0], index=data_GRD['cal_date'])
Tb_frame_GRD.columns=['TB']
Tb_frame_GRD

In [ ]:
# load dataframe of Tb SIR for plotting Tb on specified day and observation
Tb_frame=pd.DataFrame()
for i in y_dims_list:
    for j in x_dims_list:
        column=pd.DataFrame(data=data_SIR['TB'][:,i,j], columns=[str(y_s[i])+','+str(x_s[j])])
        Tb_frame=pd.concat([Tb_frame,column],axis=1)
Tb_frame.index=data_SIR['cal_date']
Tb_frame

In [ ]:
# get the min and max Tb of the 64 for each time step
Tb_64_min=Tb_frame.min(axis=1)
Tb_64_max=Tb_frame.max(axis=1)

In [ ]:
# compute std dev of the 64 SIR pixels
Tb_std_64=Tb_frame.std(axis=1)
Tb_std_64

In [ ]:
# take difference of the GRD_std_dev variable and the computed sd of the 64 SIR pixels
Tb_sd_diff=Tb_sd_frame_GRD[0]-Tb_std_64
Tb_sd_diff

In [ ]:
DAV_GRD_frame=pd.DataFrame(DAV_GRD[:,0,0], index=data_GRD['cal_date'])
DAV_GRD_frame

In [ ]:
DAV_SIR_frame=pd.DataFrame()
for i in y_dims_list:
    for j in x_dims_list:
        column=pd.DataFrame(data=DAV_SIR[:,i,j], columns=[str(y_s[i])+','+str(x_s[j])])
        DAV_SIR_frame=pd.concat([DAV_SIR_frame,column],axis=1)
DAV_SIR_frame['Date']=data_SIR['cal_date']        
DAV_SIR_frame=DAV_SIR_frame.set_index('Date')
DAV_SIR_frame

In [ ]:
DAV_nearest=DAV_SIR_frame[Site_nearest_row_col]
DAV_nearest=DAV_nearest.to_frame()
DAV_nearest.columns=['DAV']
DAV_nearest

In [ ]:
Tb_nearest=Tb_frame[Site_nearest_row_col]
Tb_nearest=Tb_nearest.to_frame()
Tb_nearest.columns=['TB']
Tb_nearest

In [ ]:
Site_nearest_row_col, sat_SIR

In [ ]:
outFile = Path(outDir, '%s_%s_%s-%sTS.png' % (sat_SIR, SiteLabel, str(subYears[0]), str(subYears[-1])))
 
#Generate a time series of specified length (any window in dataset)
#Make sure to update filename in first line to export new file
fig,(ax1,ax2,ax3)=plt.subplots(3,1, sharex=True, figsize=(10,10))

ax1.set_title(str(lat_start)+'_'+str(lon_start)+' '+Site)
#Tb_sd_frame_GRD.plot(xlim=['2019-01-01','2019-12-31'])

ax1.plot(Tb_64_max, color='red', label='SIR-Tb-max')
ax1.plot(Tb_64_min, color='blue', label='SIR-Tb-min')
ax1.plot(Tb_nearest, color = 'black',label='SIR-Site')
ax1.plot(Tb_frame_GRD, color='#636363', label='GRD-Tb')
ax1.set_ylabel('Tb (K)', fontsize=16)
ax1.set_ylim(200,300)
#ax1.yticks(fontsize=12)
ax1.legend(fontsize=12)
ax2.plot(Tb_sd_frame_GRD, color='black', label='GRD-StDev')
ax2.plot(Tb_std_frame[Site_nearest_row_col], color='#fdbb84', label='SIR-1px')
ax2.set_ylabel('Tb-STD (K)',fontsize=16)
ax2.plot(Tb_std_64, color='#4292c6',label='SIR-64px-StDev')
ax2.plot(Tb_sd_diff, label='StDevDiff_GRD-64SIR', color='green')
ax2.legend(fontsize=10,loc='lower right')
ax2.set_ylim(-12,30)
ax3.plot(DAV_GRD_frame, color='black', label='GRD-DAV')
ax3.plot(DAV_SIR_frame[Site_nearest_row_col], color='#4292c6', label='SIR-DAV')
ax3.set_ylabel('DAV (K)',fontsize=16)
ax3.legend(fontsize=12)
ax3.set_ylim(0,50)
#ax3.set_xlim('2022-01-01', '2022-10-31') #

#ax1.set_xlim('2010-01-01', '2011-12-31') #there is something wrong with this line (new since 10-26-21) 
#fix above line probably changing date format
#related to matplotlib version maybe
fig.autofmt_xdate()
plt.savefig(outFile)


In [ ]:
outFile = Path(outDir, '%s-SSMIstackeddav%s-%s.png' % (SiteLabel, str(subYears[0]), str(subYears[-1])))

#time range default Jan 1 - Dec 31, 1 year
#subset of a year is a shorter value
startmonth= 1
startday=1
endmonth=12
endday=31

#generate a plot with stacked DAV
fig,ax=plt.subplots(len(subYears),1, figsize=(10,14))
fig.suptitle(str(lat_start)+'_'+str(lon_start)+' '+Site, fontweight='bold')

for i, (yr, df) in enumerate(DAV_SIR_frame.groupby(DAV_SIR_frame.index.year)):
    #print(i,yr)
    
    df.plot(ax=ax[i],y=[Site_nearest_row_col], color='#4292c6')#, #label=str(yr)+' SIR-DAV')

    
    startdt=pd.to_datetime('%04d%02d%02d' % (yr, startmonth, startday), format='%Y%m%d')
    enddt=pd.to_datetime('%04d%02d%02d' % (yr, endmonth, endday), format='%Y%m%d')
    
    ax[i].set_ylabel('DAV (K)')#fontsize=16, #fontweight='bold')
    ax[i].set_ylim(0,50)
    ax[i].axhline(y=DAV_threshold, color='gray', linestyle='dashed')
    #ax.tick_params(right= True,top= True,left= True, bottom= True)
    #ax[i].set_xlabel('month')
    #ax[i].tick_params(labelbottom=False)
    ax[i].set_xlim(startdt, enddt)
    #ax[i].legend(fontsize=12)
    
fig.tight_layout()
    
plt.savefig(outFile)

In [ ]:
Tb_sd_frame_GRD

In [ ]:
#working
#I made this work for the GRD SD but what I really want to do is create
# a secondary axis and show DAV and SD on different axes
#attempts are commented out because I obviously don't totally understand the loop

outFile = Path(outDir, '%s-SSMIstackedDAV+SD%s-%s.png' % (SiteLabel, str(subYears[0]), str(subYears[-1])))

#generate a plot with stacked DAV
fig,ax=plt.subplots(len(subYears),1, figsize=(10,14))

fig.suptitle(str(lat_start)+'_'+str(lon_start)+' '+Site, fontweight='bold')

for i, (yr, df) in enumerate(DAV_SIR_frame.groupby(DAV_SIR_frame.index.year)):

    df.plot(ax=ax[i],y=[Site_nearest_row_col], color='#4292c6')
    ax[i].legend([str(yr)+' SIR-DAV'], loc='upper left')
    ax[i].set_ylabel('SIR DAV (K)')
    ax[i].set_ylim(0,30)
    ax[i].axhline(y=DAV_threshold, color='gray', linestyle='dashed')
    
    ax_right = ax[i].twinx()
    Tb_sd_GRD_year = Tb_sd_frame_GRD[Tb_sd_frame_GRD.index.year == yr]
    Tb_sd_GRD_year.plot(ax=ax_right, color='black')
    ax_right.legend([str(yr)+' GRD StDev'], loc='upper right')
    ax_right.set_ylabel('GRD StDev (K)')#fontsize=16, #fontweight='bold')
    ax_right.set_ylim(0,10)
        
fig.tight_layout()
    
plt.savefig(outFile)

In [ ]:
outFile = Path(outDir, '%s-stackedtb2.png' % SiteLabel)

#generate a plot with stacked DAV
fig,ax=plt.subplots(len(subYears),1, figsize=(10,14))
fig.suptitle(str(lat_start)+'_'+str(lon_start)+' '+Site, fontweight='bold')

for i, (yr, df) in enumerate(DAV_nearest.groupby(DAV_nearest.index.year)):

    # Subset each dataframe for just this year

    Tb_64_max_year = Tb_64_max[Tb_64_max.index.year == yr]
    Tb_64_min_year = Tb_64_min[Tb_64_min.index.year == yr]
    Tb_GRD_year = Tb_frame_GRD[Tb_frame_GRD.index.year == yr]

    Tb_64_max_year.plot(ax=ax[i], color='red', label='SIR-Tb-max')
    Tb_64_min_year.plot(ax=ax[i], color='blue', label='SIR-Tb-min')
    Tb_GRD_year.plot(ax=ax[i], color='green', label='GRD-Tb')
    
    ax[i].set_ylabel('Tb (K)')#fontsize=16, #fontweight='bold')
    ax[i].set_ylim(150,300)
    ax[i].axhline(y=Tb_threshold, color = 'gray', linestyle ='dashed', label='Tb Threshold')
    ax[i].legend(loc='upper left')
    
    ax_right=ax[i].twinx()
    df.plot(ax=ax_right, y=['DAV'], color='black', label=['SIR-Site'])
    ax_right.set_ylabel('DAV (K)')#fontsize=16, #fontweight='bold')
    ax_right.set_ylim(0,70)
    ax_right.axhline(y=DAV_threshold, color='red', linestyle='dashed', label='DAV Threshold')
    ax_right.legend(loc='upper right')
        
fig.tight_layout()
    
plt.savefig(outFile)

In [ ]:
outFile = Path(outDir, '%s_SDdif2.png' % SiteLabel)

# histogram of the difference between GRD_Tb_std variable and the computed std of the 64 SIR pixels
#plt.close()
fig,ax=plt.subplots()
values=Tb_sd_diff.dropna().values
ax.hist(values,bins=500, label='STD_Dif GRD-64SIR')
ax.set_title(Site+' all data')
ax.axvline(x=Tb_sd_diff.mean(), color='red', linestyle='dashed', label=Tb_sd_diff.mean().round(3))
ax.legend(fontsize=12)
ax.set_xlim(-10,+10) #adding in specified limits JRM in progress
ax.set_ylabel('Frequency',fontsize=14)
ax.set_xlabel('Standard Deviation Difference (K)',fontsize=14)
plt.savefig(outFile)

In [ ]:
MOD_DOY_df['Avg']

In [ ]:
# PLOT the std dev of Tb MAP - 
date='2011-04-16' # specify date of interest
obs=0 # 0 or 1 for morning or evening observation

# load Tb_std_dev array
array=Tb_std_frame.loc[date].values
array=array[obs]
GRD=Tb_sd_frame_GRD.loc[date].values
GRD=GRD[obs]

fig = plt.figure(figsize=(8,8))
# create figure and axes instances
#fig = plt.figure()
ax = fig.add_axes([0.1,0.1,0.8,0.8])
#ax = fig.add_subplot(121)
# create polar stereographic Basemap instance.
m = Basemap(projection='stere',lon_0=lon_start,lat_0=lat_start,lat_ts=90.,\
            llcrnrlat=(lat_start-.7),urcrnrlat=(lat_start+.7),\
            llcrnrlon=(lon_start-.7),urcrnrlon=(lon_start+.7),\
            rsphere=6371200.,resolution='l',area_thresh=10000, epsg=3857)
m.drawcoastlines()
m.drawcountries()
m.drawstates()
#m.etopo(scale=5, alpha=0.7)
#m.bluemarble(scale=1)
parallels = np.arange(0.,90,0.5)
m.drawparallels(parallels,labels=[1,0,0,0],fontsize=10)
# draw meridians
meridians = np.arange(180.,360.,0.5)
m.drawmeridians(meridians,labels=[0,0,0,1],fontsize=10)
m.arcgisimage(service='World_Physical_Map', xpixels = 1500, verbose= True)


x,y=m(data_SIR['longitude'].data, data_SIR['latitude'].data)
#m.plot(x,y, 'bs', markersize=11, markeredgewidth=1, alpha=0.2)
m.scatter(x,y,c=array, s=100, marker='s',lw=0, cmap='Oranges', alpha=.8)  # cmap='Blues'
x,y=m(data_GRD['longitude'].data, data_GRD['latitude'].data)
m.plot(x,y,'ro',markersize=4,markeredgewidth=0.0)
plt.title('Tb - std dev - '+date)

# plot a red line on colorbar for the MOD for the GRD pixel
cbar=plt.colorbar()
cbar.ax.hlines(cbar.norm(GRD), 0, 1, color='green') 
#cbar.ax.clim(0,1)  #color bar limits

plt.show()

fig = plt.figure(figsize=(8,8))
ax = fig.add_axes([0.1,0.1,0.8,0.8])

# create polar stereographic Basemap instance.
m = Basemap(projection='stere',lon_0=lon_start,lat_0=lat_start,lat_ts=90.,\
            llcrnrlat=(lat_start-.7),urcrnrlat=(lat_start+.7),\
            llcrnrlon=(lon_start-.7),urcrnrlon=(lon_start+.7),\
            rsphere=6371200.,resolution='l',area_thresh=10000, epsg=3857)
m.drawcoastlines()
m.drawcountries()
m.drawstates()
#m.etopo(scale=5, alpha=0.7)
#m.bluemarble(scale=1)
parallels = np.arange(0.,90,0.5)
m.drawparallels(parallels,labels=[1,0,0,0],fontsize=10)
# draw meridians
meridians = np.arange(180.,360.,0.5)
m.drawmeridians(meridians,labels=[0,0,0,1],fontsize=10)
m.arcgisimage(service='World_Physical_Map', xpixels = 1500, verbose= True)

x,y=m(data_SIR['longitude'].data, data_SIR['latitude'].data)
#m.plot(x,y, 'bs', markersize=11, markeredgewidth=1, alpha=0.2)
m.scatter(x,y,c=MOD_DOY_df['Avg'], s=150, marker='s',lw=0,cmap='Blues', alpha=.8)
x,y=m(lon_start,lat_start)
m.plot(x,y,'ro',markersize=4,markeredgewidth=0.0)
plt.title('Tb - std dev')
plt.colorbar()
plt.show()

#plots basemap but not info
#probably needs updating if we want to keep it
outFile = Path(outDir, 'test.png')

# Plot Tb map
#date='2019-05-08' # specify date of interest
date = '2011-05-18'
obs=0 # 0 for morning or 1 for evening observation

# load Tb_std_dev array
array=Tb_frame.loc[date].values
array=array[obs]
GRD=Tb_frame_GRD.loc[date].values
GRD=GRD[obs]

fig = plt.figure(figsize=(8,8))
# create figure and axes instances
#fig = plt.figure()
ax = fig.add_axes([0.1,0.1,0.8,0.8])
#ax = fig.add_subplot(121)
# create polar stereographic Basemap instance.
m = Basemap(projection='stere',lon_0=lon_start,lat_0=lat_start,lat_ts=90.,\
            llcrnrlat=(lat_start-.7),urcrnrlat=(lat_start+.7),\
            llcrnrlon=(lon_start-.7),urcrnrlon=(lon_start+.7),\
            rsphere=6371200.,resolution='l',area_thresh=10000, epsg=3857)
m.drawcoastlines()
m.drawcountries()
m.drawstates()
#m.etopo(scale=5, alpha=0.7)
#m.bluemarble(scale=1)
parallels = np.arange(0.,90,0.5)
m.drawparallels(parallels,labels=[1,0,0,0],fontsize=10)
# draw meridians
meridians = np.arange(180.,360.,0.5)
m.drawmeridians(meridians,labels=[0,0,0,1],fontsize=10)
m.arcgisimage(service='World_Physical_Map', xpixels = 1500, verbose= True)

x,y=m(data_SIR['longitude'].data, data_SIR['latitude'].data)
#m.plot(x,y, 'bs', markersize=11, markeredgewidth=1, alpha=0.2)
m.scatter(x,y,c=array, s=100, marker='s',lw=0, cmap='Greys', alpha=.8)  # cmap='Blues'
#x,y=m(ISAlons,ISAlats)
#m.plot(x,y,'ro',markersize=2,markeredgewidth=0.0)
plt.title('Tb_'+date+' '+str(obs))

# plot a red line on colorbar for the MOD for the GRD pixel
cbar=plt.colorbar()
cbar.ax.hlines(cbar.norm(GRD), 0, 1, color='red') 
plt.clim(230,250)  #color bar limits


plt.savefig(outFile)
plt.show()

#needs updating 
outFile = Path(outDir, 'testtestTB2.png')

ax = fig.add_axes([0.1,0.1,0.8,0.8])

# create polar stereographic Basemap instance.
m = Basemap(projection='stere',lon_0=lon_start,lat_0=lat_start,lat_ts=90.,\
            llcrnrlat=(lat_start-.7),urcrnrlat=(lat_start+.7),\
            llcrnrlon=(lon_start-.7),urcrnrlon=(lon_start+.7),\
            rsphere=6371200.,resolution='l',area_thresh=10000, epsg=3857)
m.drawcoastlines()
m.drawcountries()
m.drawstates()
#m.etopo(scale=5, alpha=0.7)
#m.bluemarble(scale=1)
parallels = np.arange(0.,90,0.5)
m.drawparallels(parallels,labels=[1,0,0,0],fontsize=10)
# draw meridians
meridians = np.arange(180.,360.,0.5)
m.drawmeridians(meridians,labels=[0,0,0,1],fontsize=10)
m.arcgisimage(service='World_Physical_Map', xpixels = 1500, verbose= True)

x,y=m(data_SIR['longitude'].data, data_SIR['latitude'].data)
#m.plot(x,y, 'bs', markersize=11, markeredgewidth=1, alpha=0.2)
m.scatter(x,y,c=MOD_DOY_df['Avg'], s=150, marker='s',lw=0,cmap='Blues', alpha=.8)
#x,y=m(ISAlons,ISAlats)
#m.plot(x,y,'ro',markersize=2,markeredgewidth=0.0)
plt.title('Tb - std dev')
plt.colorbar()

plt.savefig(outFile)
plt.show()

In [ ]:
# returns the average Jan-Feb DAV for each pixel for each year - SIR
Jan_Feb_DAV=Winter_DAV(data_SIR['TB'], data_SIR['cal_date'], data_SIR['cal_year'], subYears, rows_cols_env)
Jan_Feb_DAV

In [ ]:
Jan_Feb_DAV_avg=Jan_Feb_DAV.mean(axis=0)
Jan_Feb_DAV_avg=Jan_Feb_DAV_avg.values
#Jan_Feb_DAV_avg=np.round(Jan_Feb_DAV_avg)
Jan_Feb_DAV_avg

In [ ]:
# returns the average Jan-Feb DAV for the GRD pixel for each year
Jan_Feb_DAV_GRD=Winter_DAV(data_GRD['TB'], data_GRD['cal_date'], data_GRD['cal_year'], subYears, rows_cols_GRD)
Jan_Feb_DAV_GRD

In [ ]:
# avg Jan-Feb DAV for all years - GRD pixel
Jan_Feb_DAV_GRD_all=np.nanmean(Jan_Feb_DAV_GRD)
Jan_Feb_DAV_GRD_all

# PLOT WINTER DAV MAP
#may need updating but IS almost WORKING 2-23-23, needs a scale bar
# if getting errors, likely because there are 0 or only 1 early melt event for the subset
fig = plt.figure(figsize=(8,8))
# create figure and axes instances
#fig = plt.figure()
ax = fig.add_axes([0.1,0.1,0.8,0.8])
#ax = fig.add_subplot(121)
# create polar stereographic Basemap instance.
m = Basemap(projection='stere',lon_0=lon_start,lat_0=lat_start,lat_ts=90.,\
            llcrnrlat=(lat_start-.7),urcrnrlat=(lat_start+.7),\
            llcrnrlon=(lon_start-.7),urcrnrlon=(lon_start+.7),\
            rsphere=6371200.,resolution='l',area_thresh=10000, epsg=3857)
m.drawcoastlines()
m.drawcountries()
m.drawstates()
#m.etopo(scale=5, alpha=0.7)
#m.bluemarble(scale=1)
parallels = np.arange(0.,90,0.5)
m.drawparallels(parallels,labels=[1,0,0,0],fontsize=10)
# draw meridians
meridians = np.arange(180.,360.,0.5)
m.drawmeridians(meridians,labels=[0,0,0,1],fontsize=10)
m.arcgisimage(service='World_Physical_Map', xpixels = 1500, verbose= True)

x,y=m(data_SIR['longitude'],data_SIR['latitude'])
#m.plot(x,y, 'bs', markersize=11, markeredgewidth=1, alpha=0.2)
m.scatter(x,y,c=Jan_Feb_DAV_avg, s=100, marker='s',lw=0,cmap='Blues', alpha=.8)
#x,y=m(ISAlons,ISAlats)
#m.plot(x,y,'ro',markersize=2,markeredgewidth=0.0)
plt.title(sensor_SIR+'-'+Site+' - DAV - (Avg '+str(subYears[0])+'-'+str(subYears[-1])+')')

# plot a red line on colorbar for the GRD mean - all years
cbar=plt.colorbar()
cbar.ax.hlines(cbar.norm(Jan_Feb_DAV_GRD_all), 0, 1, color='green') 

plt.show()

fig = plt.figure(figsize=(8,8))
ax = fig.add_axes([0.1,0.1,0.8,0.8])

# create polar stereographic Basemap instance.
m = Basemap(projection='stere',lon_0=lon_start,lat_0=lat_start,lat_ts=90.,\
            llcrnrlat=(lat_start-.7),urcrnrlat=(lat_start+.7),\
            llcrnrlon=(lon_start-.7),urcrnrlon=(lon_start+.7),\
            rsphere=6371200.,resolution='l',area_thresh=10000, epsg=3857)
m.drawcoastlines()
m.drawcountries()
m.drawstates()
#m.etopo(scale=5, alpha=0.7)
#m.bluemarble(scale=1)
parallels = np.arange(0.,90,0.5)
m.drawparallels(parallels,labels=[1,0,0,0],fontsize=10)
# draw meridians
meridians = np.arange(180.,360.,0.5)
m.drawmeridians(meridians,labels=[0,0,0,1],fontsize=10)
m.arcgisimage(service='World_Physical_Map', xpixels = 1500, verbose= True)

x,y=m(data_SIR['longitude'],data_SIR['latitude'])
#m.plot(x,y, 'bs', markersize=11, markeredgewidth=1, alpha=0.2)
m.scatter(x,y,c=MOD_DOY_df['Avg'], s=150, marker='s',lw=0,cmap='Blues', alpha=.8)
#x,y=m(ISAlons,ISAlats)
#m.plot(x,y,'ro',markersize=2,markeredgewidth=0.0)
plt.title(sensor_SIR+'-'+Site+' - DAV - (Avg '+str(subYears[0])+'-'+str(subYears[-1])+')')
plt.colorbar()
plt.show()



In [ ]:
# winter DAV histogram
plt.close()
fig,ax=plt.subplots()
ax.hist(Jan_Feb_DAV_avg)
avg=np.mean(Jan_Feb_DAV_avg)  # calculate average winter DAV all pixels
ax.axvline(x=avg, color='blue', linestyle='dashed')  # plot a line for the average winter DAV for all pixels
ax.axvline(x=Jan_Feb_DAV_GRD_all, color='red')  # plot a line for winter DAV for GRD pixel
ax.set_xlabel('DAV (K)')
ax.set_ylabel('Count')
ax.set_title('Winter (Jan-Feb) DAV')

In [ ]:
#need to run this cell for the max DAV to work
y_dims_list=list(range(len(data_SIR['TB'][0,:,0])))  # creates a list of the y-dimension pixel indices, used for plotting
x_dims_list=list(range(len(data_SIR['TB'][0,0,:])))	# creates a list of the x-dimension pixel indices, for plotting
y_s=list(range(rows_cols_env[0],rows_cols_env[1]))  # makes a list of the y(row) numbers so the dataframe of early melt events can be indexed
x_s=list(range(rows_cols_env[2],rows_cols_env[3]))  # makes a list of the x(col) numbers so the dataframe of early melt events can be indexed

In [ ]:
# create array of the max DAV for each pixel for all years
DAV_frame=pd.DataFrame()
for i in y_dims_list:
    for j in x_dims_list:
        column=pd.DataFrame(data=DAV_SIR[:,i,j], columns=[str(y_s[i])+','+str(x_s[j])])
        DAV_frame=pd.concat([DAV_frame,column],axis=1)

DAV_frame.rename_axis(columns="Row,Col", inplace=True)
DAV_frame['date'] = data_SIR['cal_date']
DAV_frame.set_index('date', inplace=True)
#DAV_frame=DAV_frame.groupby(pd.Grouper(freq='A')).max() #note to joan might also do sum for the melt flags
DAV_max_array=DAV_frame.max().values
DAV_max_array

In [ ]:
DAV_frame

In [ ]:
# create array of the max DAV for each pixel for year of interest
year=2011  #specify year
DAV_max_array_year=DAV_frame.loc[str(year)].max().values
DAV_max_array_year

In [ ]:
DAV_max_GRD=np.nanmax(DAV_GRD)
DAV_max_GRD

In [ ]:
DAV_GRD

In [ ]:
year

In [ ]:
#Note this year needs to be the same as the max dav plot below
#year=2005
DAV_max_GRD_year=np.nanmax(DAV_GRD[data_GRD['cal_year']==year])
DAV_max_GRD_year

In [ ]:
# PLOT MAX DAV MAP
#probalby neds updating in 2-23-23
#set this one up to generate a plot
#plot_type='all'
year = 2011
plot_type=year
#NOTE YEAR SPECIFIED UP A FEW LEVELS (~3)
#year=2005

if plot_type=='all':
    array=DAV_max_array
    title=sensor_SIR+'-'+Site+' - Max DAV (K) - (Avg of '+str(subYears[0])+'-'+str(subYears[-1])+')'
    GRD=DAV_max_GRD
elif plot_type==year:

    array=DAV_max_array_year
    GRD=DAV_max_GRD_year
    title=sensor_SIR+'-'+Site+' - Max DAV (K) - ('+str(year)+')'

fig = plt.figure(figsize=(8,8))
# create figure and axes instances
#fig = plt.figure()
ax = fig.add_axes([0.1,0.1,0.8,0.8])
#ax = fig.add_subplot(121)
# create polar stereographic Basemap instance.
m = Basemap(projection='stere',lon_0=lon_start,lat_0=lat_start,lat_ts=90.,\
            llcrnrlat=(lat_start-.7),urcrnrlat=(lat_start+.7),\
            llcrnrlon=(lon_start-.7),urcrnrlon=(lon_start+.7),\
            rsphere=6371200.,resolution='l',area_thresh=10000, epsg=3857)
m.drawcoastlines()
m.drawcountries()
m.drawstates()
#m.etopo(scale=5, alpha=0.7)
#m.bluemarble(scale=1)
parallels = np.arange(0.,90,0.5)
m.drawparallels(parallels,labels=[1,0,0,0],fontsize=10)
# draw meridians
meridians = np.arange(180.,360.,0.5)
m.drawmeridians(meridians,labels=[0,0,0,1],fontsize=10)
m.arcgisimage(service='World_Physical_Map', xpixels = 1500, verbose= True)

x,y=m(data_SIR['longitude'],data_SIR['latitude'])
      # formerly pixel_lons, pixel_lats)
#m.plot(x,y, 'bs', markersize=11, markeredgewidth=1, alpha=0.2)
m.scatter(x,y,c=array, s=100, marker='s',lw=0,cmap='Blues', alpha=.8)
#x,y=m(ISAlons,ISAlats)
#m.plot(x,y,'ro',markersize=2,markeredgewidth=0.0)
plt.title(title)

# plot a red line on colorbar for the GRD max
cbar=plt.colorbar()
#cbar.ax.hlines(cbar.norm(DAV_max_GRD), 0, 1, color='red') 
cbar.ax.hlines(cbar.norm(DAV_max_GRD_year), 0, 1, color='red')
plt.show()

fig = plt.figure(figsize=(8,8))
ax = fig.add_axes([0.1,0.1,0.8,0.8])

# create polar stereographic Basemap instance.
m = Basemap(projection='stere',lon_0=lon_start,lat_0=lat_start,lat_ts=90.,\
            llcrnrlat=(lat_start-.7),urcrnrlat=(lat_start+.7),\
            llcrnrlon=(lon_start-.7),urcrnrlon=(lon_start+.7),\
            rsphere=6371200.,resolution='l',area_thresh=10000, epsg=3857)
m.drawcoastlines()
m.drawcountries()
m.drawstates()
#m.etopo(scale=5, alpha=0.7)
#m.bluemarble(scale=1)
parallels = np.arange(0.,90,0.5)
m.drawparallels(parallels,labels=[1,0,0,0],fontsize=10)
# draw meridians
meridians = np.arange(180.,360.,0.5)
m.drawmeridians(meridians,labels=[0,0,0,1],fontsize=10)
m.arcgisimage(service='World_Physical_Map', xpixels = 1500, verbose= True)

x,y=m(data_SIR['longitude'],data_SIR['latitude'])
#m.plot(x,y, 'bs', markersize=11, markeredgewidth=1, alpha=0.2)
m.scatter(x,y,c=MOD_DOY_df['Avg'], s=150, marker='s',lw=0,cmap='Blues', alpha=.8)
#x,y=m(ISAlons,ISAlats)
#m.plot(x,y,'ro',markersize=2,markeredgewidth=0.0)
plt.title(title)
plt.colorbar()
plt.show()

## Histograms

In [ ]:
year=2011
outFile = Path(outDir, '%shist%s%s.png' % (sat_SIR, str(year), SiteLabel))

# generate histogram - SIR - pick year 

Tb_threshold=252  #specify threshold be careful it was set above by sensor
data = data_SIR['TB'][data_SIR['cal_year']==year] # CETB_data for all pixels in subset
data = data[data>=0]
bins = range(150,300)  # bins for histogram
fig,ax=plt.subplots()
ax.axvline(x=Tb_threshold, color='red')
ax.hist(data, bins)
ax.set_title(str(year)+ ' '+Site+' SIR Histogram')
#ax.set_title(prefix+'row'+str(x)+'col'+str(y))
ax.set_xlabel('Brightness Temp (K)')

plt.savefig(outFile)

In [ ]:
outFile = Path(outDir, '%s_histsirall%s.png' % (SiteLabel, sat_SIR))

# generate histogram - SIR - all data

#Tb_threshold=252  #set above by sensor
data = data_SIR['TB'][:] # CETB_data for all pixels in subset
data = data[data>=0]
bins = range(150,300)  # bins for histogram
fig,ax=plt.subplots()
ax.hist(data, bins)
ax.set_title('All data SIR Histogram'+' '+Site)
ax.axvline(x=Tb_threshold, color='red')
#ax.set_title(prefix+'row'+str(x)+'col'+str(y))
ax.set_xlabel('Brightness Temp (K)')

plt.savefig(outFile)

In [ ]:
outFile = Path(outDir, '%s_HistGRD%s-%s.png' % (SiteLabel, str(subYears[0]), str(subYears[-1])))
               
# generate histogram - GRD - all data
data = data_GRD['TB'][:] # CETB_data for all pixels in subset
data = data[data>=0]
bins = range(150,300)  # bins for histogram
fig,ax=plt.subplots()
ax.hist(data, bins)
ax.set_title('All data GRD Histogram')
ax.axvline(x=Tb_threshold, color='red')
#ax.set_title(prefix+'row'+str(x)+'col'+str(y))
ax.set_xlabel('Brightness Temp (K)')

plt.savefig(outFile)

In [ ]:
# min/max/avg histogram of the 64 SIR pixels, plus the GRD pixel
year=2011
title=str(lat_start)+'_'+str(lon_start)
frame=min_max_series(data_GRD['TB'], data_SIR['TB'], data_SIR['cal_date'], data_SIR['cal_year'], year, title)
frame['date'] = data_SIR['cal_date']
frame.set_index('date', inplace=True)
frame

In [ ]:
#some kind of numpy pandas compatibility issue
frame.index
frame['min'].plot.hist()

In [ ]:
outFile = Path(outDir, '%s_SIRhistdis%syear.png' % (SiteLabel, str(subYears[-1] - subYears[0] + 1)))
frame.plot.hist(alpha=0.8, bins=100, histtype='step', ylim=[0,100], xlim=[150,320], title=title)
plt.savefig(outFile)

In [ ]:
year=2011
outFile = Path(outDir, '%s_tsplot.png' % SiteLabel)

# min/max/avg time series  of the 64 SIR pixels, plus the GRD pixel
# plot time series of min/max/mean of the 3 km pixels in subset and the 25km (GRD) pixel that envelopes them    
frame.plot(xlim=[str(year)+'-01-01',str(year)+'-12-31'], ylim=[150,320], title=title)
plt.savefig(outFile)

## TIME SERIES

In [ ]:
# Tb and DAV time series for GRD pixel, pick one year
year=2010
#Tb_threshold=252
#DAV_threshold=18
TbDAV_series_one_year(data_GRD['TB'],DAV_GRD, data_GRD['cal_date'], data_GRD['cal_year'], year, Tb_threshold, DAV_threshold)

In [ ]:
# Tb and DAV time series for SIR pixels, pick one year
#year=2003
#Tb_threshold=252
#DAV_threshold=18
#outFile = Path(outDir, 'SevZ_AWStimeseriesALLSIRplots2000.png')
TbDAV_series_one_year(data_SIR['TB'], DAV_SIR, data_SIR['cal_date'], data_SIR['cal_year'], year, Tb_threshold, DAV_threshold)
#plt.savefig(outFile)

In [ ]:
# Tb and DAV time series of the single SIR pixel closest to the user-specified lat/lon at top of notebook
year=2011
#Tb_threshold=252
#DAV_threshold=18
data=read_Tb_whole(datadir_SIR, prefix_SIR, subYears,
                   y_start=rows_cols[0],
                   y_end=rows_cols[1],
                   x_start=rows_cols[2],
                   x_end=rows_cols[3])

SIR_1px=data['TB']
DAV_1px=calc_DAV(SIR_1px)
TbDAV_series_one_year(SIR_1px, DAV_1px, data['cal_date'], data['cal_year'], year, Tb_threshold, DAV_threshold) 

## CREATE CSVs

In [ ]:
TB_SIR_1px=pd.DataFrame(SIR_1px.flatten(), 
                        columns=[Site_nearest_row_col], 
                        index=data['cal_date'])
TB_SIR_1px

In [ ]:
DAV_SIR_1px=pd.DataFrame(DAV_1px.flatten(),
                         columns=[Site_nearest_row_col],
                         index=data['cal_date'])
DAV_SIR_1px

In [ ]:
data['cal_date'], data_GRD['cal_date']

In [ ]:
# CREATE CSV of GRD and SIR TB, GRD and SIR64 ST, and GRD And SIR DAV

# CREATE CSV OF TB, DAV, SD and sirSD FOR THE GRD PIXEL - ALL YEARS
Tb_frame = pd.DataFrame(data_GRD['TB'][:,0,0], index=data_GRD['cal_date'])
DAV_frame = pd.DataFrame(DAV_GRD[:,0,0], index=data_GRD['cal_date'])
GRD_data_out = pd.concat([Tb_frame,DAV_frame,TB_SIR_1px,DAV_SIR_1px,Tb_sd_frame_GRD,Tb_std_64,Tb_sd_diff],
                         axis=1)
GRD_data_out.columns = ['GRDTb36V','GRDDAV36V','SIRTB36V','SIRDAV36V',
                        'GRD_SD_36V','SIR64_to_SD_36V','SD_diff[GRD-SIR]']
GRD_data_out

In [ ]:
outFile = Path(outNoisy, 'sd', 
               sat_GRD+'_'+sensor_GRD+'-'+channel_GRD+'-'+alg_GRD+'_'
               +str(lat_start)+'_'+str(lon_start)+'DATA.csv')  # modify destination to save csv
GRD_data_out.to_csv(outFile)
outFile

In [ ]:
# CREATE CSV OF TB FOR THE GRD PIXEL - ALL YEARS
Tb_frame=pd.DataFrame(data_GRD['TB'][:,0,0], index=data_GRD['cal_date'])
outFile = Path(outNoisy, 'noisytb_csv',
               sat_GRD+'_'+sensor_GRD+'-'+channel_GRD+'-'+alg_GRD+'_'
               +str(lat_start)+'_'+str(lon_start)+'.csv')  # modify destination to save csv
Tb_frame.to_csv(outFile)
outFile

In [ ]:
# CREATE CSV of DAV FOR THE GRD PIXEL - ALL YEARS
DAV_frame=pd.DataFrame(DAV_GRD[:,0,0], index=data['cal_date'])
outFile = Path(outNoisy, 'noisydav_csv',
               sat_GRD+'_'+sensor_GRD+'-'+channel_GRD+'-'+alg_GRD+'_'
               +str(lat_start)+'_'+str(lon_start)+'.csv') # modify destination to save csv
DAV_frame.to_csv(outFile)
outFile

In [ ]:
# CREATE CSV of TB FOR 64pixel subset OF SIR PIXELS - ALL YEARS
Tb_frame=pd.DataFrame()
for i in y_dims_list:
    for j in x_dims_list:
        column=pd.DataFrame(data=data_SIR['TB'][:,i,j], columns=[str(y_s[i])+','+str(x_s[j])])
        Tb_frame=pd.concat([Tb_frame,column],axis=1)
Tb_frame=Tb_frame.set_index(data_SIR['cal_date'])
outFile = Path(outNoisy, 'noisytbSIR_csv',
               sat_GRD+'_'+sensor_GRD+'-'+channel_GRD+'-'+alg_GRD+'_'
               +str(lat_start)+'_'+str(lon_start)+'.csv')  # modify destination to save csv
Tb_frame.to_csv(outFile)
outFile

In [ ]:
# CREATE CSV of DAV FOR 64pixel subset OF SIR PIXELS - ALL YEARS
DAV_frame=pd.DataFrame()
for i in y_dims_list:
    for j in x_dims_list:
        column=pd.DataFrame(data=DAV_SIR[:,i,j], columns=[str(y_s[i])+','+str(x_s[j])])
        DAV_frame=pd.concat([DAV_frame,column],axis=1)
DAV_frame=DAV_frame.set_index(data_SIR['cal_date'])
outFile = Path(outNoisy, 'noisydavSIR_csv',
               sat_GRD+'_'+sensor_GRD+'-'+channel_GRD+'-'+alg_GRD+'_'
               +str(lat_start)+'_'+str(lon_start)+'.csv')  # modify destination to save csv
DAV_frame.to_csv(outFile)
outFile
